# Pull data from database

## packages needed

In [1]:
import sqlite3
import pandas as pd

## Main code

In [2]:
# connect to the database and extract the table we need 
sqlite_file = 'files/database.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.execute("drop table NewTable")
c.execute("select name from sqlite_master where type='table' order by name")
table_name = c.fetchall()[0][0]

In [3]:
# get the full content of the database table
sql = "select * from "+table_name+";"
c.execute(sql)
full_content = c.fetchall()

In [4]:
# get table description
c.execute("PRAGMA table_info(hist)")
description = c.fetchall()

In [5]:
# get column name
c.execute("SELECT * FROM {}".format(table_name))
col_name_list = [tuple[0] for tuple in c.description]

In [6]:
# delete unnecessary information by creating a new table
create_new_table = "create table NewTable (AUNbr text, [School Name] text, [School Number] text, [Assaults on Staff] int, [Assaults on Students] int, [Conduct OSS] real, county text, enrollment int)"
c.execute(create_new_table)

In [7]:
# select the information we need from the original table
sql = "select AUNbr, [School Name], [School Number], [Assaults on Staff], [Assaults on Students],[Conduct OSS], county, enrollment from " + table_name
c.execute(sql)
to_be_inserted = c.fetchall()

In [8]:
# insert the filtered data into the new table we create before
for i in to_be_inserted:
    insertion = "insert into NewTable values(?,?,?,?,?,?,?,?)"
    c.execute(insertion, i)
c.execute("select * from NewTable")
data = c.fetchall()

In [9]:
# store the new table into a dataframe
# fill the null value with 0 if the column is numeric type
# for other missing value, for our convenient, delete the records
headers = ["AUNbr","School Name","School Number","Assaults on Staff","Assaults on Students","Conduct OSS","county","enrollment"]
df = pd.DataFrame.from_records(data, columns = headers)
df[['Assaults on Staff', 'Assaults on Students']] = df[['Assaults on Staff', 'Assaults on Students']].fillna(0)
df.dropna(axis=0, how='any', inplace=True)

In [10]:
# save the data into a csv file
df.to_csv("files/School_Safety_Data.csv",header = True)